# Neural Networks - 12 Practicals for Colab/Jupyter (Simple Version)

*Setup: Run this first*
python
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt


---

## 1. Simple Perceptron (NumPy only)

python
import numpy as np

# Initialize
weights = np.random.randn(2)
bias = 0
learning_rate = 0.1

# Activation function
def predict(x):
    output = np.dot(x, weights) + bias
    return 1 if output >= 0 else 0

# Training data (AND gate)
X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([0, 0, 0, 1])

# Train
for epoch in range(100):
    for i in range(len(X)):
        prediction = predict(X[i])
        error = y[i] - prediction
        
        # Update weights
        weights = weights + learning_rate * error * X[i]
        bias = bias + learning_rate * error

# Test
print("AND Gate Results:")
for i in range(len(X)):
    print(f"{X[i]} -> {predict(X[i])}")


---

## 2. Multi-Layer Perceptron (NumPy only)

python
import numpy as np

# Sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Initialize weights
w1 = np.random.randn(2, 4) * 0.1  # Input to hidden
b1 = np.zeros((1, 4))
w2 = np.random.randn(4, 1) * 0.1  # Hidden to output
b2 = np.zeros((1, 1))

# Forward pass
X = np.array([[0.5, 0.3]])

# Layer 1
z1 = np.dot(X, w1) + b1
a1 = sigmoid(z1)

# Layer 2
z2 = np.dot(a1, w2) + b2
output = sigmoid(z2)

print("Output:", output)


---

## 3. MLP with Backpropagation (NumPy only)

python
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Training data (XOR)
X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([[0], [1], [1], [0]])

# Initialize weights
w1 = np.random.randn(2, 4) * 0.5
b1 = np.zeros((1, 4))
w2 = np.random.randn(4, 1) * 0.5
b2 = np.zeros((1, 1))

learning_rate = 0.5

# Training loop
for epoch in range(5000):
    # Forward pass
    z1 = np.dot(X, w1) + b1
    a1 = sigmoid(z1)
    z2 = np.dot(a1, w2) + b2
    a2 = sigmoid(z2)
    
    # Calculate error
    error = a2 - y
    
    # Backpropagation
    delta2 = error * a2 * (1 - a2)
    delta1 = np.dot(delta2, w2.T) * a1 * (1 - a1)
    
    # Update weights
    w2 = w2 - learning_rate * np.dot(a1.T, delta2)
    b2 = b2 - learning_rate * np.sum(delta2, axis=0)
    w1 = w1 - learning_rate * np.dot(X.T, delta1)
    b1 = b1 - learning_rate * np.sum(delta1, axis=0)
    
    if epoch % 1000 == 0:
        loss = np.mean((a2 - y) ** 2)
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Test
print("\nXOR Results:")
for i in range(len(X)):
    z1 = np.dot(X[i], w1) + b1
    a1 = sigmoid(z1)
    z2 = np.dot(a1, w2) + b2
    prediction = sigmoid(z2)
    print(f"{X[i]} -> {prediction[0][0]:.4f}")


---

## 4. CNN with NumPy only

python
import numpy as np

# Convolution
def conv2d(image, kernel):
    h, w = image.shape
    kh, kw = kernel.shape
    output = np.zeros((h-kh+1, w-kw+1))
    
    for i in range(output.shape[0]):
        for j in range(output.shape[1]):
            output[i,j] = np.sum(image[i:i+kh, j:j+kw] * kernel)
    return output

# ReLU activation
def relu(x):
    return np.maximum(0, x)

# Max pooling
def max_pool(image, size=2):
    h, w = image.shape
    output = np.zeros((h//size, w//size))
    
    for i in range(0, h, size):
        for j in range(0, w, size):
            output[i//size, j//size] = np.max(image[i:i+size, j:j+size])
    return output

# Test
image = np.random.randn(28, 28)
kernel = np.random.randn(3, 3)

conv_output = conv2d(image, kernel)
activated = relu(conv_output)
pooled = max_pool(activated)

print(f"Image: {image.shape}")
print(f"After convolution: {conv_output.shape}")
print(f"After pooling: {pooled.shape}")


---

## 5. CNN with TensorFlow

python
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

# Load MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 28, 28, 1) / 255.0
X_test = X_test.reshape(-1, 28, 28, 1) / 255.0

# Build CNN
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')


---

## 6. Different Optimizers

python
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.datasets import mnist

# Load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 784) / 255.0
X_test = X_test.reshape(-1, 784) / 255.0

# --- Model with SGD ---
model1 = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(784,)),
    layers.Dense(10, activation='softmax')
])
model1.compile(optimizer=SGD(learning_rate=0.01),
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])
print("Training with SGD...")
model1.fit(X_train, y_train, epochs=5, batch_size=32, verbose=1)

# --- Model with Adam ---
model2 = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(784,)),
    layers.Dense(10, activation='softmax')
])
model2.compile(optimizer=Adam(learning_rate=0.001),
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])
print("\nTraining with Adam...")
model2.fit(X_train, y_train, epochs=5, batch_size=32, verbose=1)

# --- Model with RMSprop ---
model3 = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(784,)),
    layers.Dense(10, activation='softmax')
])
model3.compile(optimizer=RMSprop(learning_rate=0.001),
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])
print("\nTraining with RMSprop...")
model3.fit(X_train, y_train, epochs=5, batch_size=32, verbose=1)


---

## 7. Transfer Learning

python
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16

# Load pre-trained VGG16
base_model = VGG16(weights='imagenet',
                   include_top=False,
                   input_shape=(224, 224, 3))

# Freeze base model
base_model.trainable = False

# Add new layers
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train
# model.fit(X_train, y_train, epochs=10)

print("Model created with pre-trained VGG16!")
model.summary()


---

## 8. DCGAN

python
from tensorflow.keras import layers, models
import numpy as np

# Generator model
generator = models.Sequential([
    layers.Dense(7*7*256, input_dim=100),
    layers.Reshape((7, 7, 256)),
    layers.Conv2DTranspose(128, 5, strides=2, padding='same'),
    layers.LeakyReLU(0.2),
    layers.Conv2DTranspose(64, 5, strides=2, padding='same'),
    layers.LeakyReLU(0.2),
    layers.Conv2DTranspose(1, 5, strides=2, padding='same', activation='tanh')
])

# Discriminator model
discriminator = models.Sequential([
    layers.Conv2D(64, 5, strides=2, padding='same', input_shape=(28,28,1)),
    layers.LeakyReLU(0.2),
    layers.Conv2D(128, 5, strides=2, padding='same'),
    layers.LeakyReLU(0.2),
    layers.Flatten(),
    layers.Dense(1, activation='sigmoid')
])

# Compile
discriminator.compile(optimizer='adam', loss='binary_crossentropy')

# Combined GAN
discriminator.trainable = False
gan = models.Sequential([generator, discriminator])
gan.compile(optimizer='adam', loss='binary_crossentropy')

print("GAN models created!")


---

## 9. Autoencoder

python
from tensorflow.keras import layers, models

# Encoder
encoder = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(784,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu')
])

# Decoder
decoder = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(32,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(784, activation='sigmoid')
])

# Complete autoencoder
autoencoder = models.Sequential([encoder, decoder])

# Compile
autoencoder.compile(optimizer='adam', loss='mse')

# Load and prepare data
from tensorflow.keras.datasets import mnist
(X_train, _), (X_test, _) = mnist.load_data()
X_train = X_train.reshape(-1, 784) / 255.0
X_test = X_test.reshape(-1, 784) / 255.0

# Train (input = output)
autoencoder.fit(X_train, X_train, epochs=10, batch_size=256, validation_data=(X_test, X_test))

# Test
compressed = encoder.predict(X_test[:10])
reconstructed = decoder.predict(compressed)
print("Compressed shape:", compressed.shape)


---

## 10. Variational Autoencoder (VAE)

python
from tensorflow.keras import layers, models
import tensorflow as tf

# Encoder
encoder_input = layers.Input(shape=(784,))
x = layers.Dense(256, activation='relu')(encoder_input)
z_mean = layers.Dense(2)(x)
z_log_var = layers.Dense(2)(x)

# Sampling
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.random.normal(shape=tf.shape(z_mean))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

# Decoder
decoder_input = layers.Input(shape=(2,))
x = layers.Dense(256, activation='relu')(decoder_input)
decoder_output = layers.Dense(784, activation='sigmoid')(x)

# Models
encoder = models.Model(encoder_input, [z_mean, z_log_var, z])
decoder = models.Model(decoder_input, decoder_output)

# VAE
vae_output = decoder(z)
vae = models.Model(encoder_input, vae_output)

# Loss
reconstruction_loss = tf.reduce_mean(tf.reduce_sum(
    tf.keras.losses.binary_crossentropy(encoder_input, vae_output), axis=1))
kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
vae_loss = reconstruction_loss + kl_loss
vae.add_loss(vae_loss)

# Compile
vae.compile(optimizer='adam')

# Load data
from tensorflow.keras.datasets import mnist
(X_train, _), _ = mnist.load_data()
X_train = X_train.reshape(-1, 784) / 255.0

# Train
vae.fit(X_train, epochs=10, batch_size=128)


---

## 11. LSTM

python
from tensorflow.keras import layers, models
import numpy as np

# Example 1: Simple LSTM for sequences
model = models.Sequential([
    layers.LSTM(64, input_shape=(10, 1)),
    layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse')

# Example 2: LSTM for text classification
model_text = models.Sequential([
    layers.Embedding(10000, 128, input_length=100),
    layers.LSTM(64),
    layers.Dense(1, activation='sigmoid')
])
model_text.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Create sample time series data
data = np.sin(np.linspace(0, 100, 1000))
X, y = [], []
for i in range(len(data) - 10):
    X.append(data[i:i+10])
    y.append(data[i+10])
X = np.array(X).reshape(-1, 10, 1)
y = np.array(y)

# Train
model.fit(X, y, epochs=10, batch_size=32)
print("LSTM trained!")


---

## 12. BiLSTM

python
from tensorflow.keras import layers, models

# BiLSTM for text classification
model = models.Sequential([
    layers.Embedding(10000, 128, input_length=100),
    layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(32)),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Model summary
model.summary()

# Train with your data
# model.fit(X_train, y_train, epochs=10, batch_size=32)


---

## Quick Tips for Colab/Jupyter:

1. *Run cells one by one* - Don't run all at once
2. *Check GPU*: Runtime → Change runtime type → GPU
3. *Save work*: File → Download → .ipynb
4. *Clear output*: Edit → Clear all outputs (before submitting)

Good luck! �